In [1]:

import os
import getpass

import os
from dotenv import load_dotenv
load_dotenv()
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_key = os.environ["AZURE_OPENAI_KEY"]
search_endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
search_key = os.environ["AZURE_SEARCH_ADMIN_KEY"]


from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI

from langchain.vectorstores.azuresearch import AzureSearch
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    PromptTemplate,
)

import re

In [2]:
from typing import TypedDict, List


class ReWOO(TypedDict):
    task: str
    plan_string: str
    steps: List
    results: dict
    result: str

# from langchain_openai import ChatOpenAI

# model = ChatOpenAI(temperature=0)

model = AzureChatOpenAI(
    deployment_name="gpt-4",
    api_key=azure_openai_key,
    azure_endpoint=azure_openai_endpoint,
    api_version="2023-09-01-preview",
    # temperature=0
)

In [18]:
prompt = """For the following task, make plans that can solve the problem step by step. For each plan, indicate \
which external tool together with tool input to retrieve evidence. You can store the evidence into a \
variable #E that can be called by later tools. (Plan, #E1, Plan, #E2, Plan, ...)

Tools can be one of the following:
(1) Google[input]: Worker that searches results from Google. Useful when you need to find short
and succinct answers about a specific topic. The input should be a search query.
(2) LLM[input]: A pretrained LLM like yourself. Useful when you need to act with general
world knowledge and common sense. Prioritize it when you are confident in solving the problem
yourself. Input can be any instruction.

For example,
Plan: Use Google to search for the 2024 Australian Open winner.
#E1 = Google[2024 Australian Open winner]

Plan: Retrieve the name of the 2024 Australian Open winner from the search results.
#E2 = LLM[What is the name of the 2024 Australian Open winner, given #E1]

Plan: Use Google to search for the hometown of the 2024 Australian Open winner.
#E3 = Google[hometown of 2024 Australian Open winner, given #E2]

Plan: Retrieve the hometown of the 2024 Australian Open winner from the search results.
#E4 = LLM[What is the hometown of the 2024 Australian Open winner, given #E3]
Begin! 
Describe your plans with rich details. Each Plan should be followed by only one #E.

Task: {task}"""

In [19]:

task = "what is the hometown of the 2024 australian open winner"

result = model.invoke(prompt.format(task=task))

In [20]:
print(result.content)


type(result) # langchain_core.messages.ai.AIMessage

Plan: Use Google to search for the 2024 Australian Open winner.
#E1 = Google["2024 Australian Open winner"]

Plan: Retrieve the name of the 2024 Australian Open winner from the search results.
#E2 = LLM["What is the name of the 2024 Australian Open winner?" given #E1]

Plan: Use Google to search for the hometown of the 2024 Australian Open winner.
#E3 = Google["hometown of " + #E2]

Plan: Retrieve the hometown of the 2024 Australian Open winner from the search results.
#E4 = LLM["What is the hometown of the 2024 Australian Open winner?" given #E3]


langchain_core.messages.ai.AIMessage

## planner / excutor

In [21]:
import re
from langchain_core.prompts import ChatPromptTemplate

# Regex to match expressions of the form E#... = ...[...]
regex_pattern = r"Plan:\s*(.+)\s*(#E\d+)\s*=\s*(\w+)\s*\[([^\]]+)\]"
prompt_template = ChatPromptTemplate.from_messages([("user", prompt)])
planner = prompt_template | model


def get_plan(state: ReWOO):
    task = state["task"]
    result = planner.invoke({"task": task})
    # Find all matches in the sample text
    matches = re.findall(regex_pattern, result.content)
    return {"steps": matches, "plan_string": result.content}

In [22]:
# tavy search XX

google_res = '[{\'url\': \'https://www.cbssports.com/tennis/news/australian-open-2024-jannik-sinner-aryna-sabalenka-crowned-as-grand-slam-singles-champions-at-melbourne-park/\', \'content\': \'2024 Australian Open odds, Sinner vs. Medvedev picks Sabalenka defeats Zheng to win 2024 Australian Open  Australian Open 2024: Jannik Sinner, Aryna Sabalenka crowned as Grand Slam singles champions at Melbourne Park  2024 Australian Open odds, Sabalenka vs. Zheng picks 2024 Australian Open odds, Medvedev vs. Zverev picks  Sinner, Sabalenka win Australian Open singles titles Sinner makes epic comeback to win Australian OpenJan 28, 2024 — Jan 28, 2024Australian Open 2024: Jannik Sinner, Aryna Sabalenka crowned as Grand Slam singles champions at Melbourne Park ... Watch Now: Jannik Sinner came\\xa0...\'}, {\'url\': \'https://en.wikipedia.org/wiki/2024_Australian_Open\', \'content\': "Contents 2024 Australian Open  The 2024 Australian Open was a Grand Slam level tennis tournament held at Melbourne Park, from 14–28 January 2024.[1]  The Australian Open total prize money for 2024 increased by 13.07% year on year to a tournament record A$86,500,000.  In the tournament\'s 119-year history, this was the first Australian Open Tennis Championships to be held on an openingNovak Djokovic was the defending men\'s singles champion. ... He was defeated in the semifinals by Jannik Sinner, who went on to beat Daniil Medvedev in a five-set\\xa0..."}, {\'url\': \'https://en.wikipedia.org/wiki/2024_Australian_Open_%E2%80%93_Men%27s_singles\', \'content\': "Contents 2024 Australian Open – Men\'s singles  The entry list was released by Tennis Australia based on the ATP rankings for the week of 4 December 2023.[15]  matches, tying the Open Era record set at the 1983 US Open.[14]  feature any of the Big Three members.[4] It was the second time Medvedev lost the Australian Open final after winningJannik Sinner defeated Daniil Medvedev in the final, 3–6, 3–6, 6–4, 6–4, 6–3, to win the men\'s singles tennis title at the 2024 Australian Open."}]'

In [30]:
def _get_current_task(state: ReWOO):
    if state["results"] is None:
        return 1
    if len(state["results"]) == len(state["steps"]):
        return None
    else:
        return len(state["results"]) + 1


def tool_execution(state: ReWOO):
    """Worker node that executes the tools of a given plan."""
    _step = _get_current_task(state)
    _, step_name, tool, tool_input = state["steps"][_step - 1]
    _results = state["results"] or {}

    for k, v in _results.items(): #iterate each key/value pair
        tool_input = tool_input.replace(k, v)


    if tool == "Google":
        result = google_res

    elif tool == "LLM":
        
        result = model.invoke(tool_input)
    else:
        raise ValueError
    _results[step_name] = str(result)
    return {"results": _results}

Solver

In [31]:
solve_prompt = """Solve the following task or problem. To solve the problem, we have made step-by-step Plan and \
retrieved corresponding Evidence to each Plan. Use them with caution since long evidence might \
contain irrelevant information.

{plan}

Now solve the question or task according to provided Evidence above. Respond with the answer
directly with no extra words.

Task: {task}
Response:"""


def solve(state: ReWOO):
    plan = ""
    for _plan, step_name, tool, tool_input in state["steps"]:
        _results = state["results"] or {}
        for k, v in _results.items():
            tool_input = tool_input.replace(k, v)
            step_name = step_name.replace(k, v)
        plan += f"Plan: {_plan}\n{step_name} = {tool}[{tool_input}]"
    prompt = solve_prompt.format(plan=plan, task=state["task"])
    result = model.invoke(prompt)
    return {"result": result.content}

define graph

In [32]:
def _route(state):
    _step = _get_current_task(state)
    if _step is None:
        # We have executed all tasks
        return "solve"
    else:
        # We are still executing tasks, loop back to the "tool" node
        return "tool"

In [33]:
from langgraph.graph import StateGraph, END

graph = StateGraph(ReWOO)
graph.add_node("plan", get_plan)
graph.add_node("tool", tool_execution)
graph.add_node("solve", solve)
graph.add_edge("plan", "tool")
graph.add_edge("solve", END)
graph.add_conditional_edges("tool", _route)
graph.set_entry_point("plan")

app = graph.compile()

In [34]:
task

'what is the hometown of the 2024 australian open winner'

In [35]:
app.stream({"task": task})

<generator object Pregel.transform at 0x000001F9FF80A0C0>

In [36]:
for s in app.stream({"task": task}):
    print(s)
    print("---")

{'plan': {'steps': [('Use Google to search for the 2024 Australian Open winner.', '#E1', 'Google', '"2024 Australian Open winner"'), ('Extract the name of the winner from the search results.', '#E2', 'LLM', '"What is the name of the 2024 Australian Open winner given #E1"'), ('Use Google to search for the hometown of the 2024 Australian Open winner.', '#E3', 'Google', '"hometown of "+#E2'), ('Extract the name of the hometown from the search results.', '#E4', 'LLM', '"What is the hometown of the 2024 Australian Open winner given #E3"')], 'plan_string': 'Plan: Use Google to search for the 2024 Australian Open winner.\n#E1 = Google["2024 Australian Open winner"]\n\nPlan: Extract the name of the winner from the search results.\n#E2 = LLM["What is the name of the 2024 Australian Open winner given #E1"]\n\nPlan: Use Google to search for the hometown of the 2024 Australian Open winner.\n#E3 = Google["hometown of "+#E2]\n\nPlan: Extract the name of the hometown from the search results.\n#E4 = L